# 데이터 수집 및 가공

#### CUAI 하계컨퍼런스용

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

# 네이버 뉴스 기사 scraping

참고:
[스크레이핑 관련1](https://jeongwookie.github.io/2019/05/31/190531-naver-main-news-crawling/), 
[스크레이핑 관련2](https://wikidocs.net/86334),  
[인코딩 관련](https://dololak.tistory.com/255)

팁: 크롬에서 fn+F12로 html보기

### 관심 주식

In [2]:
stocks = ['한전기술','셀트리온','카카오게임즈','iMBC','하이브',
          '삼성전자','현대차','HMM','대한항공','네이버','두산중공업','sk하이닉스']

### 관심 주식 url로 인코딩

후에 각 주식별로 스크레이핑 하기 위해 url로 인코딩이 필요

In [3]:
import urllib.parse

stocks_url = []

for stock in stocks:
    stocks_url.append(urllib.parse.quote(stock, encoding='euc-kr'))  # 네이버가 'euc-kr'을 씀

In [4]:
stocks_url

['%C7%D1%C0%FC%B1%E2%BC%FA',
 '%BC%BF%C6%AE%B8%AE%BF%C2',
 '%C4%AB%C4%AB%BF%C0%B0%D4%C0%D3%C1%EE',
 'iMBC',
 '%C7%CF%C0%CC%BA%EA',
 '%BB%EF%BC%BA%C0%FC%C0%DA',
 '%C7%F6%B4%EB%C2%F7',
 'HMM',
 '%B4%EB%C7%D1%C7%D7%B0%F8',
 '%B3%D7%C0%CC%B9%F6',
 '%B5%CE%BB%EA%C1%DF%B0%F8%BE%F7',
 'sk%C7%CF%C0%CC%B4%D0%BD%BA']

### 주식별 기사제목 저장하기

기간: 2021-01-01 ~ 2021-07-14

#### 데이터프레임 초기화

In [5]:
column_names = ['주식','날짜','기사 제목', '언론사', '주가변동여부', '거래량변동여부']
data = pd.DataFrame(columns = column_names)

data

,주식,날짜,기사 제목,언론사,주가변동여부,거래량변동여부


In [6]:
import re  # 문자열에서 날짜 찾기용
from tqdm import tqdm  #tqdm은 for문의 진행 정도를 시각화한다
import requests  # Scraping용
from bs4 import BeautifulSoup  # Scraping용
 

for stock_num in tqdm(range(len(stocks_url))):  # 전체 루프

    url_1 = "https://finance.naver.com/news/news_search.nhn?rcdate=&q="
    url_2 = "&x=0&y=0&sm=title.basic&pd=4"
    start_end_date = "&stDateStart=2021-01-01&stDateEnd=2021-07-14"  # 기간을 쉽게 변경할 수 있게 따로 분리해 놓음
    url_3 = "&page="

    url = url_1 + stocks_url[stock_num] + url_2 + start_end_date + url_3

    
    title_list = [] # 한 주식의 기사제목들을 저장할 리스트 초기화
    date_list = [] # 한 주식의 기사가 쓰인 날짜를 저장할 리스트 초기화
    press_list = [] # 한 주식 기사의 언론사를 저장할 리스트 초기화

    for page in range(1,201):  # 왠진 모르겠는데 200페이지 이후 더이상 새로운 기사가 안떠서 200번 루프; 개별 주식의 루프
                               # 참고로 range(1,201)은 1부터 200까지임

        url_page = url + str(page)

        response = requests.get(url_page)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            ul = soup.select_one('#contentarea_left > div.newsSchResult')
            titles = ul.select('dl > dd > a')
            dates = ul.select('dl > dd > span.wdate')
            presses = ul.select('dl > dd > span.press')

            for title in titles:
                title_list.append(title.get_text())  # 기사제목 저장

            for date in dates:
                match = re.search(r'\d{4}-\d{2}-\d{2}', str(date))
                date_list.append(match.group())  # 날짜 저장
                
            for press in presses:
                press_list.append(press.get_text().strip())  # 언론사 저장

        else : 
            print(response.status_code)
            
            
    for j in range(len(title_list)): # for문(page)로 한 주식 관련 수집된 데이터를 데이터프레임에 저장
        data=data.append({'주식': stocks[stock_num], '날짜': date_list[j], '기사 제목' : title_list[j], '언론사': press_list[j], '주가변동여부': '0', '거래량변동여부': '0'}, ignore_index=True)

100%|██████████| 12/12 [06:58<00:00, 34.91s/it]


In [7]:
data

,주식,날짜,기사 제목,언론사,주가변동여부,거래량변동여부
0,한전기술,2021-07-13,"우리기술, '한전 전력연구원'과 풍력발전 제어시스템 개발 MOU",한국경제,0,0
1,한전기술,2021-07-13,"우리기술, 한전 전력연구원과 풍력발전 제어시스템 개발 양해각서 체결",파이낸셜뉴스,0,0
2,한전기술,2021-07-13,"우리기술, 한전 전력연구원과 풍력발전 제어시스템 개발 MOU",아시아경제,0,0
3,한전기술,2021-07-13,코스피200 종목 90%가 '파란불'…한전기술·신풍제약은 강세,이데일리,0,0
4,한전기술,2021-07-09,"한전, 해상풍력 설치 10일만에 끝내는 기술 개발…세계 최초",머니투데이,0,0
...,...,...,...,...,...,...
17954,sk하이닉스,2021-01-29,"[신년사] 박정호 SK하이닉스 부회장 ""경쟁자와도 손잡고 협업해야""",연합뉴스,0,0
17955,sk하이닉스,2021-01-29,"[신년사] SK하이닉스 ""글로벌 테크 리더십 강화해 ICT 1등으로""",SBS Biz,0,0
17956,sk하이닉스,2021-01-29,[특징주] 삼성전자·SK하이닉스 신고가…반도체株 새해 첫날 강세,서울경제,0,0
17957,sk하이닉스,2021-01-29,"SK하이닉스, IT 수요 호조세 지속…목표가↑-유진",서울경제,0,0


In [8]:
data.to_csv('data_title.csv', index=False)